In [6]:
# make_create_datasets.py  v4 ─ 1/3 irregular defects
# ===============================================================
# 為 one / two / three 產生 createone / createtwo / createthree
# 污漬形狀多樣且面積 ≥ 1/3，並產生 ground_truth 二值 mask
# ===============================================================

import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import random

# ---------------------- 全域參數 ----------------------
BASE = r"C:\Users\anywhere4090\Desktop\hsnu\DDAD-main\dataset\btad"
CATEGORIES = ["one", "two", "three"]
NEW_PREFIX = "create4"
IMG_EXTS = ("*.png", "*.jpg", "*.jpeg")

# 污漬最小佔比（面積比）------------------------------
MIN_RATIO = 1 / 500           # 至少 1/3
MAX_RATIO = 2 / 500           # 最多 2/3（避免整張全黑）
# ----------------------------------------------------

def list_images(folder):
    paths = []
    for ext in IMG_EXTS:
        paths.extend(glob(os.path.join(folder, ext)))
    return sorted(paths)

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

# ---------------------- 資料增強 ----------------------
def augment_image(img):
    """旋轉、縮放、亮度、對比度（較明顯）"""
    h, w = img.shape[:2]
    angle = random.uniform(-25, 25)
    scale = random.uniform(0.85, 1.15)
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, scale)
    img = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REFLECT)

    alpha = random.uniform(0.7, 1.3)   # 對比度
    beta  = random.uniform(-40, 40)    # 亮度
    img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    return img

# ---------------------- 污漬產生 ----------------------
def random_polygon(h, w):
    """回傳隨機多邊形頂點 (≥6 點)"""
    cx, cy = random.randint(0, w), random.randint(0, h)
    num   = random.randint(6, 12)
    angles = np.sort(np.random.rand(num) * 2 * np.pi)
    radii  = np.random.randint(int(0.2*min(h,w)), int(0.5*min(h,w)), size=num)
    pts = np.stack([cx + radii*np.cos(angles), cy + radii*np.sin(angles)], axis=1)
    pts = np.clip(pts, [0,0], [w-1, h-1]).astype(np.int32)
    return pts.reshape((-1,1,2))

def random_ellipse(h, w):
    """回傳隨機橢圓參數 (center, axes, angle)"""
    cx, cy = random.randint(0, w), random.randint(0, h)
    ax1 = random.randint(int(0.15*w), int(0.4*w))
    ax2 = random.randint(int(0.15*h), int(0.4*h))
    angle = random.randint(0, 179)
    return (cx, cy), (ax1, ax2), angle

def add_defect(img):
    """
    產生不規則大範圍污漬，使 mask 面積比例介於 MIN_RATIO~MAX_RATIO
    傳回 (defect_img, mask)
    """
    h, w = img.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)

    # 迴圈直到面積達標
    while True:
        choice = random.choice(["poly", "ellipse"])
        color  = (0,0,0) if random.random() < 0.5 else (255,255,255)

        if choice == "poly":
            pts = random_polygon(h, w)
            cv2.fillPoly(img, [pts], color)
            cv2.fillPoly(mask, [pts], 255)
        else:  # ellipse
            center, axes, angle = random_ellipse(h, w)
            cv2.ellipse(img, center, axes, angle, 0, 360, color, -1)
            cv2.ellipse(mask, center, axes, angle, 0, 360, 255, -1)

        ratio = mask.sum() / 255 / (h * w)
        if ratio >= MIN_RATIO:
            if ratio > MAX_RATIO:
                # 若太大，隨機裁掉一塊
                erode_size = int(np.sqrt((ratio - MAX_RATIO) * h * w / np.pi))
                if erode_size > 0:
                    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (erode_size, erode_size))
                    mask = cv2.erode(mask, kernel, iterations=1)
                    img[mask==0] = img[mask==0]  # 僅調整 mask，不需再畫色
            break
    return img, mask

# ---------------------- 主流程 ----------------------
def process_category(cat):
    src_root = os.path.join(BASE, cat)
    dst_root = os.path.join(BASE, f"{NEW_PREFIX}{cat}")

    paths = {
        "src_train_good": os.path.join(src_root, "train", "good"),
        "src_test_good" : os.path.join(src_root, "test", "good"),
        "dst_train_good": os.path.join(dst_root, "train", "good"),
        "dst_test_good" : os.path.join(dst_root, "test", "good"),
        "dst_test_bad"  : os.path.join(dst_root, "test", "bad"),
        "dst_gt"        : os.path.join(dst_root, "ground_truth")
    }
    [ensure_dir(p) for p in paths.values() if p.startswith(dst_root)]

    # -------- train/good --------
    print(f"\n[ {cat} ] train/good → {NEW_PREFIX}{cat}/train/good")
    for idx, img_path in enumerate(tqdm(list_images(paths["src_train_good"]))):
        img = cv2.imread(img_path)
        aug = augment_image(img)
        cv2.imwrite(os.path.join(paths["dst_train_good"], f"{idx:03d}.png"), aug)

    # -------- test/good --------
    print(f"[ {cat} ] test/good → {NEW_PREFIX}{cat}/test/good")
    for idx, img_path in enumerate(tqdm(list_images(paths["src_test_good"]))):
        img = cv2.imread(img_path)
        aug = augment_image(img)
        cv2.imwrite(os.path.join(paths["dst_test_good"], f"{idx:03d}.png"), aug)

    # -------- test/bad + ground_truth --------
    print(f"[ {cat} ] defects ≥ 1/3 → {NEW_PREFIX}{cat}/test/bad + ground_truth")
    for idx, img_path in enumerate(tqdm(list_images(paths["src_test_good"]))):
        img = cv2.imread(img_path)
        defect_img, mask = add_defect(img.copy())
        cv2.imwrite(os.path.join(paths["dst_test_bad"], f"{idx:03d}.png"), defect_img)
        cv2.imwrite(os.path.join(paths["dst_gt"],      f"{idx:03d}.png"), mask)

if __name__ == "__main__":
    random.seed(42)
    np.random.seed(42)

    for cat in CATEGORIES:
        process_category(cat)

    print("\n✅ 資料集產生完成！createone / createtwo / createthree 已更新。")



[ one ] train/good → create4one/train/good


100%|██████████| 400/400 [00:33<00:00, 12.07it/s]


[ one ] test/good → create4one/test/good


100%|██████████| 21/21 [00:01<00:00, 11.22it/s]


[ one ] defects ≥ 1/3 → create4one/test/bad + ground_truth


...

[ three ] test/good → create4three/test/good


100%|██████████| 400/400 [00:06<00:00, 65.54it/s]


[ three ] defects ≥ 1/3 → create4three/test/bad + ground_truth


100%|██████████| 400/400 [00:33<00:00, 12.06it/s]


✅ 資料集產生完成！createone / createtwo / createthree 已更新。


In [7]:
# make_create_datasets_v5.py  v5 ─ 進階大範圍與複合缺陷
# ===================================================================
# 為 one / two / three 產生 createfive{cat}
# 增加多種複合缺陷：多重形狀、噪聲斑塊、裂紋線條、邊界模糊、亮度&對比度變化
# ===================================================================

import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import random

# ---------------------- 全域參數 ----------------------
BASE       = r"C:\Users\anywhere4090\Desktop\hsnu\DDAD-main\dataset\btad"
CATEGORIES = ["one", "two", "three"]
NEW_PREFIX = "create5"
IMG_EXTS   = ("*.png", "*.jpg", "*.jpeg")

# 缺陷佔比（面積比）------------------------------
MIN_RATIO = 1/500    # 最小面積比例
MAX_RATIO = 2/500    # 最大面積比例，以避免全黑
# ------------------------------------------------

def list_images(folder):
    paths = []
    for ext in IMG_EXTS:
        paths.extend(glob(os.path.join(folder, ext)))
    return sorted(paths)


def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

# ---------------------- 複合缺陷生成 ----------------------

def random_polygon_mask(h, w):
    cx, cy = random.randint(0, w), random.randint(0, h)
    num = random.randint(6, 12)
    angles = np.sort(np.random.rand(num) * 2*np.pi)
    radii = np.random.randint(int(0.2*min(h, w)), int(0.5*min(h, w)), size=num)
    pts = np.stack([cx + radii*np.cos(angles), cy + radii*np.sin(angles)], axis=1)
    pts = np.clip(pts, [0, 0], [w-1, h-1]).astype(np.int32)
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(mask, [pts.reshape((-1,1,2))], 255)
    return mask


def random_ellipse_mask(h, w):
    cx, cy = random.randint(0, w), random.randint(0, h)
    ax1 = random.randint(int(0.15*w), int(0.4*w))
    ax2 = random.randint(int(0.15*h), int(0.4*h))
    angle = random.randint(0, 179)
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.ellipse(mask, (cx, cy), (ax1, ax2), angle, 0, 360, 255, -1)
    return mask


def random_noise_blob_mask(h, w):
    noise = np.zeros((h, w), dtype=np.float32)
    cv2.randn(noise, 0, 1)
    ksize = random.choice([13, 17, 21])
    noise = cv2.GaussianBlur(noise, (ksize, ksize), 0)
    th = random.uniform(0.1, 0.5)
    _, mask = cv2.threshold(noise, th, 255, cv2.THRESH_BINARY)
    return mask.astype(np.uint8)


def random_crack_mask(h, w):
    mask = np.zeros((h, w), dtype=np.uint8)
    # 隨機多段線模擬裂紋
    pts = []
    n_pts = random.randint(3, 6)
    for _ in range(n_pts):
        pts.append((random.randint(0, w), random.randint(0, h)))
    for i in range(len(pts)-1):
        thickness = random.randint(1, 3)
        cv2.line(mask, pts[i], pts[i+1], 255, thickness)
    return mask


def add_complex_defect(img):
    h, w = img.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)

    # 產生 1~3 個子缺陷
    for _ in range(random.randint(1, 3)):
        choice = random.choice(["poly", "ellipse", "noise", "crack"])
        if choice == "poly":
            m = random_polygon_mask(h, w)
        elif choice == "ellipse":
            m = random_ellipse_mask(h, w)
        elif choice == "noise":
            m = random_noise_blob_mask(h, w)
        else:
            m = random_crack_mask(h, w)
        mask = cv2.bitwise_or(mask, m)

    # 邊界模糊與二值化
    mask = cv2.GaussianBlur(mask, (7, 7), 0)
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    # 確保面積比例合規
    ratio = mask.sum() / 255 / (h * w)
    if ratio < MIN_RATIO:
        return add_complex_defect(img)  # 面積太小，重做
    if ratio > MAX_RATIO:
        # 過大則侵蝕
        erode_size = int(np.sqrt((ratio - MAX_RATIO) * h * w / np.pi))
        if erode_size > 1:
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (erode_size, erode_size))
            mask = cv2.erode(mask, kernel, iterations=1)

    # 應用缺陷：隨機亮度&對比度變化
    alpha = random.uniform(0.3, 0.7)  # 對比度縮小
    beta  = random.uniform(-60, 60)   # 亮度變化
    img_mod = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    img[mask == 255] = img_mod[mask == 255]

    return img, mask

# ---------------------- 主流程 ----------------------

def process_category(cat):
    src_root = os.path.join(BASE, cat)
    dst_root = os.path.join(BASE, f"{NEW_PREFIX}{cat}")

    paths = {
        "src_train_good": os.path.join(src_root, "train", "good"),
        "src_test_good":  os.path.join(src_root, "test",  "good"),
        "dst_train_good": os.path.join(dst_root, "train", "good"),
        "dst_test_good":  os.path.join(dst_root, "test",  "good"),
        "dst_test_bad":   os.path.join(dst_root, "test",  "bad"),
        "dst_gt":         os.path.join(dst_root, "ground_truth"),
    }
    for p in paths.values():
        ensure_dir(p)

    # train/good 只做增強
    print(f"\n[ {cat} ] train/good → {NEW_PREFIX}{cat}/train/good")
    for idx, img_path in enumerate(tqdm(list_images(paths["src_train_good"]))):
        img = cv2.imread(img_path)
        aug = augment_image(img)
        cv2.imwrite(os.path.join(paths["dst_train_good"], f"{idx:03d}.png"), aug)

    # test/good 只做增強
    print(f"[ {cat} ] test/good → {NEW_PREFIX}{cat}/test/good")
    for idx, img_path in enumerate(tqdm(list_images(paths["src_test_good"]))):
        img = cv2.imread(img_path)
        aug = augment_image(img)
        cv2.imwrite(os.path.join(paths["dst_test_good"], f"{idx:03d}.png"), aug)

    # test/bad + ground_truth
    print(f"[ {cat} ] 複合缺陷 → {NEW_PREFIX}{cat}/test/bad + ground_truth")
    for idx, img_path in enumerate(tqdm(list_images(paths["src_test_good"]))):
        img = cv2.imread(img_path)
        defect_img, mask = add_complex_defect(img.copy())
        cv2.imwrite(os.path.join(paths["dst_test_bad"], f"{idx:03d}.png"), defect_img)
        cv2.imwrite(os.path.join(paths["dst_gt"],      f"{idx:03d}.png"), mask)

if __name__ == "__main__":
    random.seed(42)
    np.random.seed(42)

    for cat in CATEGORIES:
        process_category(cat)

    print("\n✅ 資料集產生完成！create5one / create5two / create5three 已更新。")



[ one ] train/good → create5one/train/good


100%|██████████| 400/400 [00:26<00:00, 15.03it/s]


[ one ] test/good → create5one/test/good


100%|██████████| 21/21 [00:01<00:00, 14.78it/s]


[ one ] 複合缺陷 → create5one/test/bad + ground_truth


...

[ three ] test/good → create5three/test/good


100%|██████████| 400/400 [00:05<00:00, 67.45it/s]


[ three ] 複合缺陷 → create5three/test/bad + ground_truth


100%|██████████| 400/400 [00:31<00:00, 12.71it/s]


✅ 資料集產生完成！create5one / create5two / create5three 已更新。


In [1]:
# make_create_datasets_v7.py  v7 ─ 統一目錄：good / test / ground_truth
# ==================================================================
# 為 one / two / three 產生 create7{cat}
# - good: 包含原 train & test 好樣本增強結果
# - test: 單純存放缺陷圖
# - ground_truth: 對應 test 中每張缺陷的二值 mask
# ==================================================================

import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import random

# ---------------------- 全域參數 ----------------------
BASE       = r"C:\Users\anywhere4090\Desktop\hsnu\DDAD-main\dataset\btad"
CATEGORIES = ["one", "two", "three"]
NEW_PREFIX = "create7"
IMG_EXTS   = ("*.png", "*.jpg", "*.jpeg")

# 缺陷佔比範圍
MIN_RATIO = 1/500
MAX_RATIO = 2/500

# ---------------------- 工具函式 ----------------------
def list_images(folder):
    paths = []
    for ext in IMG_EXTS:
        paths.extend(glob(os.path.join(folder, ext)))
    return sorted(paths)


def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

# ---------------------- 缺陷產生（簡化版） ----------------------
def random_polygon(h, w):
    cx, cy = random.randint(0, w), random.randint(0, h)
    num = random.randint(6, 12)
    angles = np.sort(np.random.rand(num) * 2 * np.pi)
    radii = np.random.randint(int(0.2*min(h, w)), int(0.5*min(h, w)), size=num)
    pts = np.stack([cx + radii*np.cos(angles), cy + radii*np.sin(angles)], axis=1)
    pts = np.clip(pts, [0,0], [w-1, h-1]).astype(np.int32)
    return pts.reshape((-1,1,2))


def random_ellipse(h, w):
    cx, cy = random.randint(0, w), random.randint(0, h)
    ax1 = random.randint(int(0.15*w), int(0.4*w))
    ax2 = random.randint(int(0.15*h), int(0.4*h))
    angle = random.randint(0, 179)
    return (cx, cy), (ax1, ax2), angle


def add_defect(img):
    h, w = img.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)
    # 產生多邊形或橢圓缺陷
    choice = random.choice(["poly", "ellipse"])
    if choice == "poly":
        pts = random_polygon(h, w)
        cv2.fillPoly(img, [pts], (0,0,0))
        cv2.fillPoly(mask, [pts], 255)
    else:
        center, axes, angle = random_ellipse(h, w)
        cv2.ellipse(img, center, axes, angle, 0, 360, (0,0,0), -1)
        cv2.ellipse(mask, center, axes, angle, 0, 360, 255, -1)
    # 面積控制
    ratio = mask.sum()/255/(h*w)
    # 若面積不符，重試
    if ratio < MIN_RATIO or ratio > MAX_RATIO:
        return add_defect(img)
    return img, mask

# ---------------------- 主流程 ----------------------
def process_category(cat):
    src_root = os.path.join(BASE, cat)
    dst_root = os.path.join(BASE, f"{NEW_PREFIX}{cat}")
    # 目錄準備
    paths = {
        'good':         os.path.join(dst_root, 'good'),
        'test':         os.path.join(dst_root, 'test'),
        'ground_truth': os.path.join(dst_root, 'ground_truth')
    }
    for p in paths.values():
        ensure_dir(p)

    # 1. train & test 好樣本複製到 good
    for mode in ['train', 'test']:
        src_folder = os.path.join(src_root, mode, 'good')
        print(f"[{cat}] {mode}/good → {NEW_PREFIX}{cat}/good")
        for idx, img_path in enumerate(tqdm(list_images(src_folder))):
            img = cv2.imread(img_path)
            cv2.imwrite(os.path.join(paths['good'], f"{mode}_{idx:04d}.png"), img)

    # 2. 從 test/good 生成缺陷到 test & ground_truth
    src_test_good = os.path.join(src_root, 'test', 'good')
    print(f"[{cat}] 生成缺陷 → {NEW_PREFIX}{cat}/test & ground_truth")
    for idx, img_path in enumerate(tqdm(list_images(src_test_good))):
        img = cv2.imread(img_path)
        img_def, mask = add_defect(img.copy())
        cv2.imwrite(os.path.join(paths['test'],         f"{idx:04d}.png"), img_def)
        cv2.imwrite(os.path.join(paths['ground_truth'], f"{idx:04d}.png"), mask)

if __name__ == '__main__':
    random.seed(42)
    np.random.seed(42)
    for cat in CATEGORIES:
        process_category(cat)
    print("\n✅ 資料集格式：good/test/ground_truth 已在 create7one/two/three 完成！")


[one] train/good → create7one/good


100%|██████████| 400/400 [00:23<00:00, 16.85it/s]


[one] test/good → create7one/good


100%|██████████| 21/21 [00:01<00:00, 16.62it/s]


[one] 生成缺陷 → create7one/test & ground_truth


 10%|▉         | 2/21 [00:07<01:14,  3.91s/it]


RecursionError: maximum recursion depth exceeded

In [2]:
# make_create_datasets_v8.py  v8 ─ 修正迴圈缺陷生成，避免遞迴錯誤
# ==================================================================
# 為 one / two / three 產生 create8{cat}
# - good: 包含原 train & test 好樣本
# - test: 單純存放缺陷圖
# - ground_truth: 對應 test 中每張缺陷的二值 mask
# ==================================================================

import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import random

# ---------------------- 全域參數 ----------------------
BASE       = r"C:\Users\anywhere4090\Desktop\hsnu\DDAD-main\dataset\btad"
CATEGORIES = ["one", "two", "three"]
NEW_PREFIX = "create8"
IMG_EXTS   = ("*.png", "*.jpg", "*.jpeg")

# 缺陷佔比範圍
MIN_RATIO = 1/500
MAX_RATIO = 2/500
MAX_ATTEMPTS = 10  # 最多嘗試次數

# ---------------------- 工具函式 ----------------------
def list_images(folder):
    paths = []
    for ext in IMG_EXTS:
        paths.extend(glob(os.path.join(folder, ext)))
    return sorted(paths)


def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

# ---------------------- 缺陷產生（迴圈版） ----------------------
def random_polygon(h, w):
    cx, cy = random.randint(0, w-1), random.randint(0, h-1)
    num = random.randint(6, 12)
    angles = np.sort(np.random.rand(num) * 2 * np.pi)
    radii = np.random.randint(int(0.2*min(h, w)), int(0.5*min(h, w)), size=num)
    pts = np.stack([cx + radii*np.cos(angles), cy + radii*np.sin(angles)], axis=1)
    pts = np.clip(pts, [0,0], [w-1, h-1]).astype(np.int32)
    return pts.reshape((-1,1,2))


def random_ellipse(h, w):
    cx, cy = random.randint(0, w-1), random.randint(0, h-1)
    ax1 = random.randint(int(0.15*w), int(0.4*w))
    ax2 = random.randint(int(0.15*h), int(0.4*h))
    angle = random.randint(0, 179)
    return (cx, cy), (ax1, ax2), angle


def add_defect(img):
    h, w = img.shape[:2]
    for attempt in range(MAX_ATTEMPTS):
        tmp_img = img.copy()
        mask = np.zeros((h, w), dtype=np.uint8)
        # 產生多邊形或橢圓缺陷
        if random.random() < 0.5:
            pts = random_polygon(h, w)
            cv2.fillPoly(tmp_img, [pts], (0,0,0))
            cv2.fillPoly(mask, [pts], 255)
        else:
            center, axes, angle = random_ellipse(h, w)
            cv2.ellipse(tmp_img, center, axes, angle, 0, 360, (0,0,0), -1)
            cv2.ellipse(mask, center, axes, angle, 0, 360, 255, -1)
        # 面積控制
        ratio = mask.sum() / 255 / (h * w)
        if MIN_RATIO <= ratio <= MAX_RATIO:
            return tmp_img, mask
    # 若多次嘗試仍不符合，直接回傳最後一次結果
    return tmp_img, mask

# ---------------------- 主流程 ----------------------
def process_category(cat):
    src_root = os.path.join(BASE, cat)
    dst_root = os.path.join(BASE, f"{NEW_PREFIX}{cat}")

    # 準備目錄
    paths = {
        'good':         os.path.join(dst_root, 'good'),
        'test':         os.path.join(dst_root, 'test'),
        'ground_truth': os.path.join(dst_root, 'ground_truth')
    }
    for p in paths.values():
        ensure_dir(p)

    # 1. 複製 train & test 好樣本到 good
    for mode in ['train', 'test']:
        src_folder = os.path.join(src_root, mode, 'good')
        print(f"[{cat}] {mode}/good → {NEW_PREFIX}{cat}/good")
        for idx, img_path in enumerate(tqdm(list_images(src_folder))):
            img = cv2.imread(img_path)
            out_name = f"{mode}_{idx:04d}.png"
            cv2.imwrite(os.path.join(paths['good'], out_name), img)

    # 2. 從 test/good 生成缺陷
    src_test_good = os.path.join(src_root, 'test', 'good')
    print(f"[{cat}] 生成缺陷 → {NEW_PREFIX}{cat}/test & ground_truth")
    for idx, img_path in enumerate(tqdm(list_images(src_test_good))):
        img = cv2.imread(img_path)
        img_def, mask = add_defect(img)
        name = f"{idx:04d}.png"
        cv2.imwrite(os.path.join(paths['test'], name), img_def)
        cv2.imwrite(os.path.join(paths['ground_truth'], name), mask)

if __name__ == '__main__':
    random.seed(42)
    np.random.seed(42)
    for cat in CATEGORIES:
        process_category(cat)
    print("\n✅ 資料集結構 create8one/two/three 已生成：good / test / ground_truth")


[one] train/good → create8one/good


100%|██████████| 400/400 [00:23<00:00, 16.95it/s]


[one] test/good → create8one/good


100%|██████████| 21/21 [00:01<00:00, 16.71it/s]


[one] 生成缺陷 → create8one/test & ground_truth


...

[three] test/good → create8three/good


100%|██████████| 400/400 [00:05<00:00, 76.36it/s]


[three] 生成缺陷 → create8three/test & ground_truth


100%|██████████| 400/400 [00:07<00:00, 54.70it/s]



✅ 資料集結構 create8one/two/three 已生成：good / test / ground_truth


In [4]:
# make_create_datasets_v9.py  v9 ─ 更小＆更複雜的小型多重缺陷
# ==================================================================
# 為 one / two / three 產生 create9{cat}
# - good: 包含原 train & test 好樣本
# - test: 多個微小缺陷組合，難度提升
# - ground_truth: 對應二值 mask
# ==================================================================

import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import random

# ---------------------- 全域參數 ----------------------
BASE         = r"C:\Users\anywhere4090\Desktop\hsnu\DDAD-main\dataset\btad"
CATEGORIES   = ["one", "two", "three"]
NEW_PREFIX   = "create9"
IMG_EXTS     = ("*.png", "*.jpg", "*.jpeg")

# 更小占比：0.02%~0.4%
MIN_RATIO    = 0.05
MAX_RATIO    = 0.3
MAX_ATTEMPTS = 5   # 減少嘗試次數

# ---------------------- 工具函式 ----------------------
def list_images(folder):
    paths = []
    for ext in IMG_EXTS:
        paths.extend(glob(os.path.join(folder, ext)))
    return sorted(paths)


def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

# ---------------------- 小型多重缺陷生成 ----------------------
def random_small_blob(h, w):
    x, y = random.randint(0, w-1), random.randint(0, h-1)
    r     = random.randint(1, 5)  # 小半徑
    mask  = np.zeros((h, w), dtype=np.uint8)
    cv2.circle(mask, (x, y), r, 255, -1)
    return mask


def random_line(h, w):
    # 細長裂紋
    mask = np.zeros((h, w), dtype=np.uint8)
    x1, y1 = random.randint(0, w-1), random.randint(0, h-1)
    angle  = random.uniform(0, 2*np.pi)
    length = random.randint(int(min(h,w)*0.05), int(min(h,w)*0.2))
    x2 = int(x1 + length * np.cos(angle))
    y2 = int(y1 + length * np.sin(angle))
    cv2.line(mask, (x1,y1), (x2,y2), 255, 1)
    return mask


def add_defects(img):
    h, w = img.shape[:2]
    combined_mask = np.zeros((h, w), dtype=np.uint8)
    # 隨機生成多個微小缺陷 2~5 個
    n = random.randint(2, 5)
    for _ in range(n):
        if random.random() < 0.6:
            m = random_small_blob(h, w)
        else:
            m = random_line(h, w)
        combined_mask = cv2.bitwise_or(combined_mask, m)
    # 面積檢查
    ratio = combined_mask.sum()/255/(h*w)
    # 若面積過小或過大，稍微調整：放大 mask
    if ratio < MIN_RATIO:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        combined_mask = cv2.dilate(combined_mask, kernel, iterations=1)
    elif ratio > MAX_RATIO:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        combined_mask = cv2.erode(combined_mask, kernel, iterations=1)
    # 應用到影像：缺陷處塗黑
    img_def = img.copy()
    img_def[combined_mask == 255] = (0,0,0)
    return img_def, combined_mask

# ---------------------- 主流程 ----------------------
def process_category(cat):
    src_root = os.path.join(BASE, cat)
    dst_root = os.path.join(BASE, f"{NEW_PREFIX}{cat}")
    # 準備目錄
    paths = {
        'good':         os.path.join(dst_root, 'good'),
        'test':         os.path.join(dst_root, 'test'),
        'ground_truth': os.path.join(dst_root, 'ground_truth')
    }
    for p in paths.values(): ensure_dir(p)

    # 1. 複製 train & test 好樣本到 good
    for mode in ['train', 'test']:
        folder = os.path.join(src_root, mode, 'good')
        print(f"[{cat}] {mode}/good → {NEW_PREFIX}{cat}/good")
        for i, p in enumerate(tqdm(list_images(folder))):
            img = cv2.imread(p)
            out = f"{mode}_{i:04d}.png"
            cv2.imwrite(os.path.join(paths['good'], out), img)

    # 2. 從 test/good 生成更難缺陷
    src_test = os.path.join(src_root, 'test', 'good')
    print(f"[{cat}] 生成微小多重缺陷 → {NEW_PREFIX}{cat}/test & ground_truth")
    for i, p in enumerate(tqdm(list_images(src_test))):
        img = cv2.imread(p)
        img_def, mask = add_defects(img)
        name = f"{i:04d}.png"
        cv2.imwrite(os.path.join(paths['test'], name), img_def)
        cv2.imwrite(os.path.join(paths['ground_truth'], name), mask)

if __name__ == '__main__':
    random.seed(42)
    np.random.seed(42)
    for c in CATEGORIES: process_category(c)
    print(f"\n✅ 微小多重缺陷資料集已生成 create9one/two/three：good/test/ground_truth")


[one] train/good → create9one/good


100%|██████████| 400/400 [00:23<00:00, 17.00it/s]


[one] test/good → create9one/good


100%|██████████| 21/21 [00:01<00:00, 17.43it/s]


[one] 生成微小多重缺陷 → create9one/test & ground_truth


...

[three] test/good → create9three/good


100%|██████████| 400/400 [00:05<00:00, 77.05it/s]


[three] 生成微小多重缺陷 → create9three/test & ground_truth


100%|██████████| 400/400 [00:06<00:00, 66.54it/s]


✅ 微小多重缺陷資料集已生成 create9one/two/three：good/test/ground_truth


In [5]:
# make_create_datasets_v10.py  v10 ─ 中小型多重缺陷（增大尺寸）
# ==================================================================
# 為 one / two / three 產生 create10{cat}
# - good: 包含原 train & test 好樣本
# - test: 更大範圍微小與中型缺陷混合
# - ground_truth: 對應二值 mask
# ==================================================================

import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import random

# ---------------------- 全域參數 ----------------------
BASE         = r"C:\Users\anywhere4090\Desktop\hsnu\DDAD-main\dataset\btad"
CATEGORIES   = ["one", "two", "three"]
NEW_PREFIX   = "create10"
IMG_EXTS     = ("*.png", "*.jpg", "*.jpeg")

# 占比範圍：0.1%~2%
MIN_RATIO    = 0.001
MAX_RATIO    = 0.02
MAX_ATTEMPTS = 5

# ---------------------- 工具函式 ----------------------

def list_images(folder):
    paths = []
    for ext in IMG_EXTS:
        paths.extend(glob(os.path.join(folder, ext)))
    return sorted(paths)


def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

# ---------------------- 缺陷生成函式 ----------------------

def random_blob(h, w):
    # 包含中型斑點
    x, y = random.randint(0, w-1), random.randint(0, h-1)
    r     = random.randint(5, 20)  # 中小半徑
    mask  = np.zeros((h, w), dtype=np.uint8)
    cv2.circle(mask, (x, y), r, 255, -1)
    return mask


def random_ellipse(h, w):
    # 中小型橢圓缺陷
    cx, cy = random.randint(0, w-1), random.randint(0, h-1)
    ax1 = random.randint(int(0.05*w), int(0.2*w))
    ax2 = random.randint(int(0.05*h), int(0.2*h))
    angle = random.randint(0, 179)
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.ellipse(mask, (cx, cy), (ax1, ax2), angle, 0, 360, 255, -1)
    return mask


def random_line(h, w):
    # 細長裂紋，長度擴大
    mask = np.zeros((h, w), dtype=np.uint8)
    x1, y1 = random.randint(0, w-1), random.randint(0, h-1)
    angle  = random.uniform(0, 2*np.pi)
    length = random.randint(int(min(h,w)*0.1), int(min(h,w)*0.4))
    x2 = int(x1 + length * np.cos(angle))
    y2 = int(y1 + length * np.sin(angle))
    cv2.line(mask, (x1,y1), (x2,y2), 255, random.randint(1,3))
    return mask


def add_defects(img):
    h, w = img.shape[:2]
    combined_mask = np.zeros((h, w), dtype=np.uint8)
    # 隨機生成 1~4 個缺陷
    n = random.randint(1, 4)
    for _ in range(n):
        choice = random.random()
        if choice < 0.4:
            m = random_blob(h, w)
        elif choice < 0.7:
            m = random_ellipse(h, w)
        else:
            m = random_line(h, w)
        combined_mask = cv2.bitwise_or(combined_mask, m)

    # 面積範圍檢查
    ratio = combined_mask.sum()/255/(h*w)
    if ratio < MIN_RATIO:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
        combined_mask = cv2.dilate(combined_mask, kernel, iterations=1)
    elif ratio > MAX_RATIO:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
        combined_mask = cv2.erode(combined_mask, kernel, iterations=1)

    # 應用缺陷：填黑
    img_def = img.copy()
    img_def[combined_mask == 255] = (0,0,0)
    return img_def, combined_mask

# ---------------------- 主流程 ----------------------

def process_category(cat):
    src_root = os.path.join(BASE, cat)
    dst_root = os.path.join(BASE, f"{NEW_PREFIX}{cat}")
    paths = {
        'good':         os.path.join(dst_root, 'good'),
        'test':         os.path.join(dst_root, 'test'),
        'ground_truth': os.path.join(dst_root, 'ground_truth')
    }
    for p in paths.values(): ensure_dir(p)

    # 複製好樣本
    for mode in ['train', 'test']:
        for i, p in enumerate(tqdm(list_images(os.path.join(src_root, mode, 'good')))):
            img = cv2.imread(p)
            cv2.imwrite(os.path.join(paths['good'], f"{mode}_{i:04d}.png"), img)

    # 生成缺陷
    for i, p in enumerate(tqdm(list_images(os.path.join(src_root, 'test', 'good')))):
        img = cv2.imread(p)
        img_def, mask = add_defects(img)
        name = f"{i:04d}.png"
        cv2.imwrite(os.path.join(paths['test'], name), img_def)
        cv2.imwrite(os.path.join(paths['ground_truth'], name), mask)

if __name__ == '__main__':
    random.seed(42); np.random.seed(42)
    for c in CATEGORIES: process_category(c)
    print(f"\n✅ 中小型多重缺陷資料集 create10one/two/three 已生成：good/test/ground_truth")


100%|██████████| 400/400 [00:06<00:00, 66.62it/s]


✅ 中小型多重缺陷資料集 create10one/two/three 已生成：good/test/ground_truth
